In [1]:
import socnet as sn
import networkx as nx
import numpy as np
from pprint import pprint
import scipy.stats as ss
import math

In [2]:
g = sn.load_graph('./one-mode.gml')
sn.graph_width = 720
sn.graph_height = 480

sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'
sn.show_graph(g, nlab=True)

/Users/raphacosta/Desktop/Desktop - MacBook Pro do Bruce/Insper6/RedesSociais/myRepo/redesoc-2018/HubseAuthorities/socnet.py:59: RuntimeWarning:

divide by zero encountered in double_scalars

/Users/raphacosta/Desktop/Desktop - MacBook Pro do Bruce/Insper6/RedesSociais/myRepo/redesoc-2018/HubseAuthorities/socnet.py:187: RuntimeWarning:

invalid value encountered in double_scalars

/Users/raphacosta/Desktop/Desktop - MacBook Pro do Bruce/Insper6/RedesSociais/myRepo/redesoc-2018/HubseAuthorities/socnet.py:188: RuntimeWarning:

invalid value encountered in double_scalars

/Users/raphacosta/Desktop/Desktop - MacBook Pro do Bruce/Insper6/RedesSociais/myRepo/redesoc-2018/HubseAuthorities/socnet.py:189: RuntimeWarning:

invalid value encountered in double_scalars

/Users/raphacosta/Desktop/Desktop - MacBook Pro do Bruce/Insper6/RedesSociais/myRepo/redesoc-2018/HubseAuthorities/socnet.py:190: RuntimeWarning:

invalid value encountered in double_scalars



In [3]:
nodes = []
for k, v in dict(g.in_degree()).items():
    g.nodes[k]['in_degree'] = v
    nodes.append(g.nodes[k])
    
sorted(nodes, key= lambda x: x['in_degree'])

[{'label': 'Joao',
  'color': (255, 255, 255),
  'pos': (0.372613848068914, 0.738914072670916),
  'in_degree': 0},
 {'label': 'Paulo',
  'color': (255, 255, 255),
  'pos': (0.4395118777334555, 0.6593569285108623),
  'in_degree': 0},
 {'label': 'Ale',
  'color': (255, 255, 255),
  'pos': (0.0, 0.7022799271918745),
  'in_degree': 0},
 {'label': 'Pedro',
  'color': (255, 255, 255),
  'pos': (0.48254872513367336, 0.7409197816421325),
  'in_degree': 0},
 {'label': 'Tirta',
  'color': (255, 255, 255),
  'pos': (0.7178382073255338, 0.7114885077930985),
  'in_degree': 0},
 {'label': 'Jean',
  'color': (255, 255, 255),
  'pos': (0.0796876138760869, 0.7329783073557956),
  'in_degree': 1},
 {'label': 'Bruna',
  'color': (255, 255, 255),
  'pos': (0.6561213732736184, 0.5493815980018053),
  'in_degree': 1},
 {'label': 'Carlos',
  'color': (255, 255, 255),
  'pos': (0.561761106116478, 0.489295369739165),
  'in_degree': 1},
 {'label': 'Leo',
  'color': (255, 255, 255),
  'pos': (0.9323458960851876, 0

In [4]:
one_mode_absolut_vote_order = list(reversed(sorted([g.nodes[x] for x in g.nodes], key=lambda x: x['in_degree'])))
one_mode_absolut_vote_order_names = [(x['label'], x['in_degree']) for x in one_mode_absolut_vote_order]
one_mode_absolut_vote_order_names

[('Hugo', 12),
 ('Graicer', 10),
 ('Forba', 7),
 ('Toyama', 7),
 ('Fred', 7),
 ('Chen', 6),
 ('Vitoria', 5),
 ('Rachel', 5),
 ('Isa', 3),
 ('Antonio', 3),
 ('Rapha', 3),
 ('Martim', 2),
 ('Rios', 2),
 ('Sabrina', 2),
 ('Elisa', 2),
 ('Gabs', 2),
 ('Leo', 2),
 ('Carlos', 1),
 ('Bruna', 1),
 ('Jean', 1),
 ('Tirta', 0),
 ('Pedro', 0),
 ('Ale', 0),
 ('Paulo', 0),
 ('Joao', 0)]

In [5]:
g2 = sn.load_graph('./two-mode.gml')
generos = ['Samba', 'MPB', 'Rap Nacional', 'Opera', 'Rock classico', 'Jazz', 'Pop', 'K-Pop', 'J-Pop', 'Pagode', 'Metal', 'Indie Rock', 'Funk', 'Sertanejo', 'Eletronica', 'Pop Rock', 'Disco']

# two_mode = {}
# soma = 0

# Calcula in degrees dos generos
for k, indeg in dict(g2.in_degree()).items():
    if g2.nodes[k]['label'] in generos:
        g2.nodes[k]['is_genre'] = True
        g2.nodes[k]['indegree_ponderado'] = indeg
    else:
        g2.nodes[k]['is_genre'] = False
        g2.nodes[k]['outdegree_ponderado'] = 0

euclid_anterior = 0
euclid_atual = 0
delta = 1

while delta > 0.000001:
    vec = []
    # Calcula outdegree        
    for k, v in dict(g2.nodes).items():
        if not v['is_genre']:
            succ = list(g2.successors(k))
            for s in succ:
                g2.nodes[k]['outdegree_ponderado'] += g2.nodes[s]['indegree_ponderado']

    # Calcula indegree ponderado
    for k, v in dict(g2.nodes).items():
        if v['is_genre']:
            predec = list(g2.predecessors(k))
            for p in predec:
                g2.nodes[k]['indegree_ponderado'] += g2.nodes[p]['outdegree_ponderado']

    # Normalizar
    soma_in = sum([g2.nodes[x]['indegree_ponderado'] for x in g2.nodes if g2.nodes[x]['is_genre']])
    soma_out = sum([g2.nodes[x]['outdegree_ponderado'] for x in g2.nodes if not g2.nodes[x]['is_genre']])

    for k, v in dict(g2.nodes).items():
        if (v['is_genre']):
            g2.nodes[k]['indegree_ponderado'] /= soma_in
            vec.append(g2.nodes[k]['indegree_ponderado'])
        else:
            g2.nodes[k]['outdegree_ponderado'] /= soma_out
            vec.append(g2.nodes[k]['outdegree_ponderado'])
    
    euclid_atual = np.array(vec)
    euclid_atual = np.sum(euclid_atual**2)**0.5
    delta = abs(euclid_atual - euclid_anterior)
    euclid_anterior = euclid_atual

## Hubs

In [6]:
hubs = list(reversed(sorted([g2.nodes[x] for x in g2.nodes if not g2.nodes[x]['is_genre']], key=lambda x: x['outdegree_ponderado'])))
hubs

[{'label': 'Forba',
  'color': (255, 255, 255),
  'pos': (0.5085503976841529, 0.5100853822962302),
  'is_genre': False,
  'outdegree_ponderado': 0.09457695040590426},
 {'label': 'Jean',
  'color': (255, 255, 255),
  'pos': (0.5303580665458026, 0.37330640440917845),
  'is_genre': False,
  'outdegree_ponderado': 0.06976807569750428},
 {'label': 'Sabrina',
  'color': (255, 255, 255),
  'pos': (0.5507993407874793, 0.42914054293180726),
  'is_genre': False,
  'outdegree_ponderado': 0.06845074980707899},
 {'label': 'Toyama',
  'color': (255, 255, 255),
  'pos': (0.5248737402822721, 0.4806414463809527),
  'is_genre': False,
  'outdegree_ponderado': 0.05998733471247563},
 {'label': 'Rachel',
  'color': (255, 255, 255),
  'pos': (0.4158828940500143, 0.5085013613297089),
  'is_genre': False,
  'outdegree_ponderado': 0.05404076765751253},
 {'label': 'Vitoria',
  'color': (255, 255, 255),
  'pos': (0.4854448832504887, 0.36325505665128827),
  'is_genre': False,
  'outdegree_ponderado': 0.0537447663

## Authorities

In [7]:
# Authorities
auths = list(reversed(sorted([g2.nodes[x] for x in g2.nodes if g2.nodes[x]['is_genre']], key=lambda x: x['indegree_ponderado'])))
auths

[{'label': 'Eletronica',
  'color': (255, 255, 255),
  'pos': (0.7051415865049973, 0.01852423808928649),
  'is_genre': True,
  'indegree_ponderado': 0.12792875764859404},
 {'label': 'Rock classico',
  'color': (255, 255, 255),
  'pos': (0.5770438063969432, 0.00832147154839616),
  'is_genre': True,
  'indegree_ponderado': 0.08027758814996976},
 {'label': 'Pop Rock',
  'color': (255, 255, 255),
  'pos': (0.0472483050781352, 0.7877495505539458),
  'is_genre': True,
  'indegree_ponderado': 0.07817693860604079},
 {'label': 'Pop',
  'color': (255, 255, 255),
  'pos': (0.8343231428998901, 0.11627402364370279),
  'is_genre': True,
  'indegree_ponderado': 0.07446041825185795},
 {'label': 'Pagode',
  'color': (255, 255, 255),
  'pos': (0.0, 0.6216416390785363),
  'is_genre': True,
  'indegree_ponderado': 0.07205255698074242},
 {'label': 'Rap Nacional',
  'color': (255, 255, 255),
  'pos': (0.6593336238790795, 1.0),
  'is_genre': True,
  'indegree_ponderado': 0.06778848912149975},
 {'label': 'Ind

## Hubs e Authorities com peso

In [8]:
g3 = sn.load_graph('./one-mode.gml')
for n in g3.nodes:
    g3.nodes[n]['weight_received_votes'] = 0
    g3.nodes[n]['received_votes'] = len(list(g3.predecessors(n)))
    g3.nodes[n]['vote_weight'] = 1 / (len(list(g3.successors(n)))) if len(list(g3.successors(n))) != 0 else 0

for n in g3.nodes:
    pessoa = g3.nodes[n]
    sucessores = g3.successors(n)
    for s in sucessores:
        g3.nodes[s]['weight_received_votes'] += pessoa['received_votes'] * pessoa['vote_weight']


In [9]:
g3.nodes[18]

{'label': 'Graicer',
 'color': (255, 255, 255),
 'pos': (0.17580907489060707, 0.15537470727310737),
 'weight_received_votes': 10.86111111111111,
 'received_votes': 10,
 'vote_weight': 0.5}

In [10]:
len(list(g.successors(18)))

2

Pesos considerados: <br></br>
Valor do voto: (1 / total de votos da pessoa) * número de votos recebidos
<h5> Ex: Se uma pessoa votou em 5 pessoas e recebeu 2 votos, seu voto vale (1/5) * 2 = 0.4</h5>

In [11]:
for node in g3.nodes:
    print(g3.nodes[node])

{'label': 'Rachel', 'color': (255, 255, 255), 'pos': (0.5448246744409391, 0.5684988275470656), 'weight_received_votes': 4.711111111111111, 'received_votes': 5, 'vote_weight': 0.25}
{'label': 'Jean', 'color': (255, 255, 255), 'pos': (0.23845843564561672, 0.5360655308329212), 'weight_received_votes': 0.0, 'received_votes': 1, 'vote_weight': 1.0}
{'label': 'Bruna', 'color': (255, 255, 255), 'pos': (0.2343533962633151, 0.4516634657167697), 'weight_received_votes': 1.25, 'received_votes': 1, 'vote_weight': 0.1111111111111111}
{'label': 'Leo', 'color': (255, 255, 255), 'pos': (0.1782575751027912, 0.0), 'weight_received_votes': 0.1111111111111111, 'received_votes': 2, 'vote_weight': 1.0}
{'label': 'Joao', 'color': (255, 255, 255), 'pos': (0.33328612942573793, 0.41951877636512647), 'weight_received_votes': 0, 'received_votes': 0, 'vote_weight': 0.16666666666666666}
{'label': 'Rapha', 'color': (255, 255, 255), 'pos': (0.15053211409506223, 0.19598724651133023), 'weight_received_votes': 1.5999999

In [12]:
sorted_by_weight_received_votes = list(reversed(sorted([g3.nodes[x] for x in g3.nodes], key=lambda x: x['weight_received_votes'])))
sorted_by_weight_received_votes_names = [(x['label'], round(x['weight_received_votes'], 2)) for x in sorted_by_weight_received_votes]
sorted_by_weight_received_votes_names

[('Hugo', 15.25),
 ('Graicer', 10.86),
 ('Forba', 9.81),
 ('Fred', 7.56),
 ('Isa', 5.45),
 ('Toyama', 5.11),
 ('Chen', 5.0),
 ('Vitoria', 4.76),
 ('Rachel', 4.71),
 ('Sabrina', 3.25),
 ('Elisa', 2.11),
 ('Rios', 1.65),
 ('Rapha', 1.6),
 ('Bruna', 1.25),
 ('Martim', 1.2),
 ('Antonio', 0.61),
 ('Carlos', 0.5),
 ('Gabs', 0.2),
 ('Leo', 0.11),
 ('Tirta', 0),
 ('Pedro', 0),
 ('Ale', 0),
 ('Paulo', 0),
 ('Joao', 0),
 ('Jean', 0.0)]

In [13]:
def get_node_by_name(name):
    return list(filter(lambda x: x['label'] == name, sorted_by_weight_received_votes))[0] or False

In [14]:
g4 = sn.load_graph('./two-mode.gml')
generos = ['Samba', 'MPB', 'Rap Nacional', 'Opera', 'Rock classico', 'Jazz', 'Pop', 'K-Pop', 'J-Pop', 'Pagode', 'Metal', 'Indie Rock', 'Funk', 'Sertanejo', 'Eletronica', 'Pop Rock', 'Disco']

# Calcula in degrees dos generos
for k, v in dict(g4.nodes).items():
    if g4.nodes[k]['label'] in generos:
        g4.nodes[k]['is_genre'] = True
        g4.nodes[k]['indegree_ponderado'] = 0
        predecessors = g4.predecessors(k)
        for p in predecessors:
            pred = get_node_by_name(g4.nodes[p]['label'])
            pred_successors = len(list(g4.successors(p)))
            g4.nodes[k]['indegree_ponderado'] += pred['weight_received_votes'] / pred_successors
            
    else:
        g4.nodes[k]['is_genre'] = False
        g4.nodes[k]['outdegree_ponderado'] = 0

euclid_anterior = 0
euclid_atual = 0
delta = 1

while delta > 0.000001:
    vec = []
    # Calcula outdegree        
    for k, v in dict(g4.nodes).items():
        if not v['is_genre']:
            succ = list(g4.successors(k))
            for s in succ:
                g4.nodes[k]['outdegree_ponderado'] += g4.nodes[s]['indegree_ponderado']

    # Calcula indegree ponderado
    for k, v in dict(g4.nodes).items():
        if v['is_genre']:
            predec = list(g4.predecessors(k))
            for p in predec:
                pred_successors = len(list(g4.successors(p)))
                g4.nodes[k]['indegree_ponderado'] += g4.nodes[p]['outdegree_ponderado'] / pred_successors

    # Normalizar
    soma_in = sum([g4.nodes[x]['indegree_ponderado'] for x in g4.nodes if g4.nodes[x]['is_genre']])
    soma_out = sum([g4.nodes[x]['outdegree_ponderado'] for x in g4.nodes if not g4.nodes[x]['is_genre']])

    for k, v in dict(g4.nodes).items():
        if (v['is_genre']):
            g4.nodes[k]['indegree_ponderado'] /= soma_in
            vec.append(g4.nodes[k]['indegree_ponderado'])
        else:
            g4.nodes[k]['outdegree_ponderado'] /= soma_out
            vec.append(g4.nodes[k]['outdegree_ponderado'])
    
    euclid_atual = np.array(vec)
    euclid_atual = np.sum(euclid_atual**2)**0.5
    delta = abs(euclid_atual - euclid_anterior)
    euclid_anterior = euclid_atual

In [15]:
i = 0
for x in g4.nodes:
    pprint(g4.nodes[x])
    print(i)
    i+=1

{'color': (255, 255, 255),
 'indegree_ponderado': 0.019264505751136612,
 'is_genre': True,
 'label': 'Samba',
 'pos': (0.7303987509068234, 0.9508483373865899)}
0
{'color': (255, 255, 255),
 'is_genre': False,
 'label': 'Isa',
 'outdegree_ponderado': 0.05153129453735397,
 'pos': (0.5984823142873914, 0.6660653180143282)}
1
{'color': (255, 255, 255),
 'is_genre': False,
 'label': 'Chen',
 'outdegree_ponderado': 0.043263889430431246,
 'pos': (0.474292869483784, 0.42404037027119756)}
2
{'color': (255, 255, 255),
 'indegree_ponderado': 0.04737040980319133,
 'is_genre': True,
 'label': 'MPB',
 'pos': (0.014576247765078817, 0.3687854921205953)}
3
{'color': (255, 255, 255),
 'indegree_ponderado': 0.07800693720294961,
 'is_genre': True,
 'label': 'Rap Nacional',
 'pos': (0.5937488041582206, 0.0)}
4
{'color': (255, 255, 255),
 'indegree_ponderado': 0.08288355399447726,
 'is_genre': True,
 'label': 'Pop',
 'pos': (0.6941307456194795, 0.06343859724998367)}
5
{'color': (255, 255, 255),
 'is_genre': 

In [16]:
len(list(g4.successors(39)))

6

In [17]:
auths_weighted = list(reversed(sorted([g4.nodes[x] for x in g4.nodes if g4.nodes[x]['is_genre']], key=lambda x: x['indegree_ponderado'])))
print('   AUTHS            AUTHS_WEIGHTED\n')
for i in range(len(auths)):
    print(f'{i+1}  '[0:2], '{0: <16}'.format(auths[i]['label']), auths_weighted[i]['label'])

   AUTHS            AUTHS_WEIGHTED

1  Eletronica       Eletronica
2  Rock classico    Pop
3  Pop Rock         Pop Rock
4  Pop              Rock classico
5  Pagode           Rap Nacional
6  Rap Nacional     Pagode
7  Indie Rock       Funk
8  Disco            Sertanejo
9  Sertanejo        Disco
10 Funk             Indie Rock
11 MPB              Metal
12 Metal            MPB
13 J-Pop            J-Pop
14 Jazz             Jazz
15 Samba            Samba
16 K-Pop            K-Pop
17 Opera            Opera


In [32]:
hubs_weighted = list(reversed(sorted([g4.nodes[x] for x in g4.nodes if not g4.nodes[x]['is_genre']], key=lambda x: x['outdegree_ponderado'])))
print('   HUBS                           HUBS_WEIGHTED\n')
# print('   HUBS       ABSOLUT_IN_DEGREE\n')
for i in range(len(hubs)):
#     print(f'{i+1}  '[0:2], '{0: <10}'.format(hubs[i]['label']),  one_mode_absolut_vote_order_names[i])
    print(f'{i+1}  '[0:2], '{0: <10}'.format(hubs[i]['label']), hubs[i]['outdegree_ponderado'], hubs_weighted[i]['label'], hubs_weighted[i]['outdegree_ponderado'] )

   HUBS                           HUBS_WEIGHTED

1  Forba      0.09457695040590426 Forba 0.08987359562756422
2  Jean       0.06976807569750428 Sabrina 0.06670355153443042
3  Sabrina    0.06845074980707899 Jean 0.0661613673589607
4  Toyama     0.05998733471247563 Toyama 0.05931434540371955
5  Rachel     0.05404076765751253 Rachel 0.05747732200492138
6  Vitoria    0.05374476636202175 Vitoria 0.052741628259137266
7  Hugo       0.051297512289265726 Hugo 0.05197070393892891
8  Isa        0.050524660640416816 Isa 0.05153129453735397
9  Graicer    0.04723974395057163 Joao 0.04907040404428515
10 Joao       0.04671934985942606 Graicer 0.04681212297863892
11 Fred       0.0444191537727633 Martim 0.04449840579621194
12 Chen       0.04399605317137955 Chen 0.043263889430431246
13 Pedro      0.04199038158921065 Rapha 0.04141124316451644
14 Martim     0.04097677898963952 Gabs 0.041411243164516436
15 Gabs       0.03859748313531624 Fred 0.041378657429530084
16 Rapha      0.03859748313531623 Pedro 0.0406

## Regressão linear indegrees do one mode x hubs two-mode

In [19]:
def print_r(r):
    print("slope: ", r[0])
    print("intercept: ", r[1])
    print("rvalue: ", r[2])
    print("pvalue: ", r[3])
    print("stderr: ", r[4])

In [20]:
def get_one_mode_node_by_name(label):
    for i in range(len(g.nodes)):
        if(g.nodes[i]['label'] == label):
            return g.nodes[i]

In [21]:
two_mode_names = list([g2.nodes[x]['label'] for x in g2.nodes if not g2.nodes[x]['is_genre']])
two_mode_outdegrees = list([g2.nodes[x]['outdegree_ponderado'] for x in g2.nodes if not g2.nodes[x]['is_genre']])

In [22]:
one_mode_nodes = []
for i in range(len(two_mode_names)):
    one_mode_nodes.append(get_one_mode_node_by_name(two_mode_names[i]))

In [23]:
one_mode_names = list([x['label'] for x in one_mode_nodes])
one_mode_in_degrees = list([x['in_degree'] for x in one_mode_nodes])

In [24]:
r = ss.linregress(two_mode_outdegrees, one_mode_in_degrees)
print_r(r)

slope:  66.17812669742635
intercept:  0.644429274024942
rvalue:  0.3914935536910227
pvalue:  0.06469449062062638
stderr:  33.94324803881003


## Regressão linear hubs sem peso x hubs com peso

In [25]:
outdegree_hubs_weighted = list([g4.nodes[x]['outdegree_ponderado'] for x in g4.nodes if not g4.nodes[x]['is_genre']])
outdegree_hubs = list([g2.nodes[x]['outdegree_ponderado'] for x in g2.nodes if not g2.nodes[x]['is_genre']])
r = ss.linregress(outdegree_hubs_weighted, outdegree_hubs)
print_r(r)

slope:  1.0391395120071347
intercept:  -0.0017017179133536828
rvalue:  0.9943171635032275
pvalue:  6.437164623996206e-22
stderr:  0.024278338883083266


## Regressão linear hubs com peso x in degree one mode

In [33]:
outdegree_hubs_weighted = list([g4.nodes[x]['outdegree_ponderado'] for x in g4.nodes if not g4.nodes[x]['is_genre']])
r = ss.linregress(outdegree_hubs_weighted, one_mode_in_degrees)
print_r(r)

slope:  68.97524878973194
intercept:  0.5228152700116557
rvalue:  0.39044017524118757
pvalue:  0.06548165431347697
stderr:  35.4906089347281


## Regressão linear auths sem peso x auths com peso

In [34]:
indegrees_auths = list([g2.nodes[x]['indegree_ponderado'] for x in g2.nodes if g2.nodes[x]['is_genre']])
indegrees_auths_weighted = list([g4.nodes[x]['indegree_ponderado'] for x in g4.nodes if g4.nodes[x]['is_genre']])
r = ss.linregress(indegrees_auths, indegrees_auths_weighted)
print_r(r)

slope:  1.1719943036121738
intercept:  -0.01011731197718671
rvalue:  0.9781910593968535
pvalue:  1.1936132631149662e-11
stderr:  0.06425509713013375


## Explicando terceira métrica

A terceira métrica utilizada foi a atribuição de pesos aos votos individuais nos genêros disponíveis. Esta métrica foi criada para tentar evitar que uma pessoa que votasse em todos ou muitos genêros fosse considerada um bom recomendador (alto valor de hub). Da mesma forma, desejávamos valorizar os votos de pessoas que receberam vários votos, com base na pesquisa inicial do gráfo one-mode. Portanto, para penalizar aqueles que votaram em várias pessoas na primeira pesquisa, dividimos o peso de seu voto, que inicialmente valia 1, pelo número de votos que essa pessoa realizou. Ao mesmo tempo, para beneficiar aqueles que receberam votos, multiplicamos os pesos dos votos individuais pelo número de votos recebidos para cada indivíduo. Por exemplo, Graicer inicialmente tinha um voto de valor 1; Pelo fato de ter votado em 2 pessoas, teve seu valor de voto reduzido para 0.5. Porém, como foi votado por 10 pessoas, seu voto, ao final, vale 5.
$$Peso = \dfrac{1}{T} * R$$
    $$T: \text{Total de votos concedidos}$$ 
    $$R: \text{Total de votos recebidos}$$ 
    
Assim, este valor será a base para o cálculo do voto de uma pessoa à um genêro na segunda etapa. Este voto terá peso igual ao $Peso$ correspondente divido pelo número total de votos concedidos, para genêros, por aquele indivíduo. Este processo foi efetuado na segunda pesquisa, buscando penalizar aqueles em que votaram em vários genêros. Seguindo o mesmo exemplo, quando Graicer vota em um genêro, seu voto na verdade vale $\dfrac{5}{6}$.
$$\text{Peso Genêro} = \dfrac{Peso}{T}$$
$$T: \text{Total de votos concedidos à genêros}$$

## Calculando centralities

In [ ]:
nx.local_reaching_centrality

In [ ]:
for i in range(len(g.nodes)):
    g.nodes[i]['reaching'] = nx.local_reaching_centrality(g, i)

## Regressão reaching x in_degree

In [ ]:
reaching = list([g.nodes[x]['reaching'] for x in g.nodes])
indegrees = list([g.nodes[x]['in_degree'] for x in g.nodes])
r = ss.linregress(reaching, indegrees)
print_r(r)

## Regressão reaching x hubs

In [ ]:
outdegree_hubs = list([g2.nodes[x]['outdegree_ponderado'] for x in g2.nodes if not g2.nodes[x]['is_genre']])
names_hubs = list([g2.nodes[x]['label'] for x in g2.nodes if not g2.nodes[x]['is_genre']])
reaching_filter = list([g.nodes[x]['reaching'] for x in g.nodes if g.nodes[x]['label'] in names_hubs])
r = ss.linregress(reaching_filter, outdegree_hubs)
print_r(r)